In [101]:
import random
import itertools
import os
import math
from functools import partial

from meb import utils
from meb import datasets
from meb import core
from meb.utils.traditional_methods import tim

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import f1_score
from skimage.transform import resize
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from torchvision import transforms
from PIL import Image
from tqdm import tqdm

pd.set_option('display.max_columns', 50)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [191]:
c = datasets.CrossDataset(resize=112, color=True, preload=False)
df = c.data_frame
data = c.data

In [192]:
n_frames = 50
pr_frames = []
for i, video in tqdm(enumerate(data), total=len(data)):
    video = tim(video.transpose(1, 2, 3, 0), n_frames)
    video = resize(video, (124, 124, 3, 50))
    video = video.transpose(2, 3, 0, 1)
    pr_frames.append(video)

100%|███████████████████████████████████████| 2031/2031 [49:54<00:00,  1.47s/it]


In [193]:
from functools import partial

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm3d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3x3(planes, planes)
        self.bn2 = nn.BatchNorm3d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x
        
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample is not None:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)

        return out
    
    
def conv3x3x3(in_planes, out_planes, stride=1):
    # 3x3x3 convolution with padding
    return nn.Conv3d(
        in_planes,
        out_planes,
        kernel_size=3,
        stride=stride,
        padding=1,
        bias=False)


class Covpool(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        x = input

        batchSize = x.data.shape[0]
        dim = x.data.shape[1]
        h = x.data.shape[2]
        w = x.data.shape[3]
        M = h*w
        x = x.reshape(batchSize,dim,M)
        I_hat = (-1./M/M)*torch.ones(M,M,device = x.device) + (1./M)*torch.eye(M,M,device = x.device)
        I_hat = I_hat.view(1,M,M).repeat(batchSize,1,1).type(x.dtype)
        y = x.bmm(I_hat).bmm(x.transpose(1,2))
        ctx.save_for_backward(input,I_hat)
        return y
    @staticmethod
    def backward(ctx, grad_output):
        input,I_hat = ctx.saved_tensors
        x = input
        batchSize = x.data.shape[0]
        dim = x.data.shape[1]
        h = x.data.shape[2]
        w = x.data.shape[3]
        M = h*w
        x = x.reshape(batchSize,dim,M)
        grad_input = grad_output + grad_output.transpose(1,2)
        grad_input = grad_input.bmm(x).bmm(I_hat)
        grad_input = grad_input.reshape(batchSize,dim,h,w)
        return grad_input

    
def cov_feature(x):
    batchsize = x.data.shape[0]
    dim = x.data.shape[1]
    h = x.data.shape[2]
    w = x.data.shape[3]
    M = h*w
    x = x.reshape(batchsize,dim,M)
    I_hat = (-1./M/M)*torch.ones(dim,dim,device = x.device) + (1./M)*torch.eye(dim,dim,device = x.device)
    I_hat = I_hat.view(1,dim,dim).repeat(batchsize,1,1).type(x.dtype)
    y = (x.transpose(1,2)).bmm(I_hat).bmm(x)
    return y


def downsample_basic_block(x, planes, stride):
    out = F.avg_pool3d(x, kernel_size=1, stride=stride)
    if isinstance(out.data, torch.cuda.FloatTensor):
        zero_pads = out.new(
            out.size(0), planes - out.size(1), out.size(2), out.size(3),
            out.size(4)).zero_()
    #zero_pads = torch.Tensor(
    #    out.size(0), planes - out.size(1), out.size(2), out.size(3),
    #    out.size(4)).zero_()
    #if isinstance(out.data, torch.cuda.FloatTensor):
    #    zero_pads = zero_pads.to(device)

    out = torch.autograd.Variable(torch.cat([out.data, zero_pads], dim=1))

    return out

    
def CovpoolLayer(var):
    return Covpool.apply(var)


class ResNet_multiple(nn.Module):

    def __init__(self, block, layers, sample_size, sample_duration,
                 shortcut_type='A', num_classes=2):
        self.inplanes = 64
        super(ResNet_multiple, self).__init__()
        self.conv1 = nn.Conv3d(3, 64, kernel_size=7, stride=(1, 2, 2),
            padding=(3, 3, 3), bias=False)
        self.bn1 = nn.BatchNorm3d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool3d(kernel_size=(3, 3, 3), stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0], shortcut_type)
        self.layer2 = self._make_layer(
            block, 128, layers[1], shortcut_type, stride=2)
        self.layer3 = self._make_layer(
            block, 256, layers[2], shortcut_type, stride=2)
        self.layer4 = self._make_layer(
            block, 512, layers[3], shortcut_type, stride=2)
        
        ############################################################
        
        last_duration = int(math.ceil(sample_duration / 10))
        last_size = int(math.ceil(sample_size / 28))
        self.tanh=nn.Tanh()
        self.avgpool = nn.AvgPool3d(
            (last_duration, last_size, last_size), stride=1)

        self.avgpool2d = nn.AvgPool2d(
            (last_size, last_size), stride=1)

        fc_num = 512
        self.fc = nn.Linear(fc_num, num_classes)
        self.isqrt_dim = 16
 
        self.layer_reduce = nn.Conv2d(512, self.isqrt_dim, kernel_size=1, stride=1, padding=0,
                                          bias=False)
        self.layer_reduce_bn = nn.BatchNorm2d(self.isqrt_dim)
        self.layer_reduce_relu = nn.ReLU(inplace=True)
      

        last_size = 4
        self.sp_reso = last_size * last_size
        self.row_bn_for_spatial = nn.BatchNorm2d(self.sp_reso)
        self.relu_normal = nn.ReLU(inplace=False)
        self.row_conv_group_for_spatial = nn.Conv2d( 
                 self.sp_reso, self.sp_reso*4, kernel_size=(self.sp_reso, 1), 
                 groups=self.sp_reso, bias=True)
        self.fc_adapt_channels_for_spatial = nn.Conv2d(
                 self.sp_reso * last_size, self.sp_reso, kernel_size=1, groups=1, bias=True)
        self.sigmoid = nn.Sigmoid()
        
        self.ch_dim = 512  #512 2048
        self.expansion = 1 #1
        planes = 512  #512 2048
        self.row_bn = nn.BatchNorm2d(self.expansion * self.ch_dim)
        self.row_conv_group = nn.Conv2d(
             self.ch_dim, self.ch_dim, 
             kernel_size=(self.ch_dim, 1), 
             groups = self.ch_dim, bias=True)
        self.fc_adapt_channels = nn.Conv2d(
             self.ch_dim, self.expansion*self.ch_dim, 
             kernel_size=1, groups=1, bias=True)
        self.sigmoid = nn.Sigmoid()

        self.conv_for_DR = nn.Conv2d(
             planes * self.expansion, self.ch_dim, 
             kernel_size=1,stride=2, bias=True)
        self.bn_for_DR = nn.BatchNorm2d(self.expansion*self.ch_dim)
        self.row_bn = nn.BatchNorm2d(self.expansion*self.ch_dim)
            #row-wise conv is realized by group conv

        #####cooncat###############
        self.groups_base = 32
        self.groups = int(planes * self.expansion / 64)
        self.factor = int(math.log(self.groups_base / self.groups, 2))
        self.padding_num = self.factor + 2
        self.conv_kernel_size = self.factor * 2 + 5
        self.dilate_conv_for_concat1 = nn.Conv2d(planes * self.expansion, 
                                                planes * self.expansion, 
                                                kernel_size=(self.conv_kernel_size,1), 
                                                stride=1, padding=(self.padding_num,0),
                                                groups=self.groups, bias=True)
        
        self.dilate_conv_for_concat2 = nn.Conv2d(planes * self.expansion, 
                                                planes * self.expansion, 
                                                kernel_size=(self.conv_kernel_size,1), 
                                                stride=1, padding=(self.padding_num,0),
                                                groups=self.groups, bias=True)

        self.bn_for_concat = nn.BatchNorm2d(planes * self.expansion)         
        #####cooncat###############
        #self.fc = nn.Linear(512, num_classes)     
        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                m.weight = nn.init.kaiming_normal_(m.weight, mode='fan_out')
            elif isinstance(m, nn.BatchNorm3d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()


    def _make_layer(self, block, planes, blocks, shortcut_type, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            if shortcut_type == 'A':
                downsample = partial(
                    downsample_basic_block,
                    planes=planes * block.expansion,
                    stride=stride)
            else:
                downsample = nn.Sequential(
                    nn.Conv3d(
                        self.inplanes,
                        planes * block.expansion,
                        kernel_size=1,
                        stride=stride,
                        bias=False), nn.BatchNorm3d(planes * block.expansion))

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    
    def chan_att(self, out):
        out = CovpoolLayer(out) # Nxdxd
        out = out.view(out.size(0), out.size(1), out.size(2), 1).contiguous() # Nxdxdx1
        out = self.row_bn(out)
        out = self.row_conv_group(out) # Nx512x1x1
        out = self.sigmoid(out) #NxCx1x1
        return out


    def pos_att(self, out):
        out = cov_feature(out) # Nx16x16
        out = out.view(out.size(0), out.size(1), out.size(2), 1).contiguous()  # Nx16x16x1
        out = self.row_bn_for_spatial(out)

        out = self.row_conv_group_for_spatial(out) # Nx256x1x1
        out = self.relu(out)

        out = self.fc_adapt_channels_for_spatial(out) #Nx64x1x1
        out = self.sigmoid(out) 
        out = out.view(out.size(0), 1, 4, 4).contiguous()#Nx1x8x8
        return out


    def downsample_long_block(self, x, planes, stride):
        out = F.avg_pool3d(x, kernel_size=1, stride=stride)
        zero_pads = torch.Tensor(
            out.size(0), planes - out.size(1), out.size(2), out.size(3),
            out.size(4)).zero_()
        if isinstance(out.data, torch.cuda.FloatTensor):
            zero_pads = zero_pads.to(device)

        out = Variable(torch.cat([out.data, zero_pads], dim=1))

        return out


    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        residual0 = x
        x = self.layer1(x)
        residual1 = x
        x = self.layer2(x)
        residual2 = x
        x = self.layer3(x)
        x = self.layer4(x)
        x=x.squeeze(2)

        pre_att = x
        chan_att = self.chan_att(x)
        pos_att = self.pos_att(x)
        out1 = self.dilate_conv_for_concat1(pre_att * chan_att)
        out2 = self.dilate_conv_for_concat2(self.relu(pre_att * pos_att))
        out = (out1 * pre_att * chan_att) + out2 * (self.relu(pre_att * pos_att))
        x = self.bn_for_concat(out)
        x = self.avgpool(x)
        feature_map = x
        
        x = x.view(x.size(0), -1)

        x = self.fc(x)
        return x#, feature_map
    
    
def resnet_multiple18(**kwargs):
    """Constructs a ResNet-18 model.
    """
    model = ResNet_multiple(BasicBlock, [2, 2, 2, 2], **kwargs)
    return model

In [194]:
def get_fine_tuning_parameters(model, ft_begin_index):
    if ft_begin_index == 0:
        return model.parameters()

    ft_module_names = []
    for i in range(ft_begin_index, 5):
        ft_module_names.append('layer{}'.format(i))

    ft_module_names.append('fc')
    parameters = []
    for k, v in model.named_parameters():
        for ft_module in ft_module_names:
            if ft_module in k:
                parameters.append({'params': v})
                break
        else:
            parameters.append({'params': v, 'lr': 0.0001})

    return parameters

In [195]:
import pytorchvideo
spatial_train_transform = transforms.Compose([
    pytorchvideo.transforms.RandomResizedCrop(112, 112, [1, 1], [1, 1]),
    transforms.RandomHorizontalFlip(0.5),
])

spatial_test_transform = transforms.Compose([
    transforms.Resize(112),
])

In [196]:
def get_model(num_classes):
    net = resnet_multiple18(num_classes=num_classes, sample_size=112, sample_duration=10)
    pretrained_model = torch.load("../data/resnet-18-kinetics-ucf101_split1.pth")["state_dict"]
    net_state_dict = net.state_dict()
    new_state_dict = {k[7:]: v for k, v in pretrained_model.items() if k[7:] in net_state_dict.keys()}
    del new_state_dict["fc.bias"]
    del new_state_dict["fc.weight"]
    net_state_dict.update(new_state_dict)
    net.load_state_dict(net_state_dict)
    return net

In [197]:
class Config(core.Config):
    device = torch.device("cuda:1")
    epochs = 40
    evaluation_fn = [
        partial(utils.MultiLabelF1Score, average="macro"),
        partial(utils.MultiLabelF1Score, average="binary"),
    ]
    batch_size = 32
    optimizer = partial(optim.SGD, lr=0.001, weight_decay=0.001, momentum=0.9)
    train_transform = {
        "spatial": spatial_train_transform,
        "temporal": datasets.NoisyUniformTemporalSubsample(10),
    }
    test_transform = {
        "spatial": spatial_test_transform,
        "temporal": datasets.UniformTemporalSubsample(10),
    }
    model = partial(get_model, num_classes=len(core.Config.action_units))
    

In [198]:
input_data = np.array(pr_frames).transpose(0, 2, 3, 4, 1)

In [199]:
out = SCAValidator(Config).validate(df, input_data)

100%|███████████████████████████████████████████| 40/40 [22:12<00:00, 33.31s/it]


MultiLabelF1Score
Dataset: casme, n=189 | train_mean: 0.9668 | test_mean: 0.6187
Test per AU: [('AU1', 80.32), ('AU2', 80.71), ('AU4', 74.69), ('AU5', 47.65), ('AU6', 49.87), ('AU7', 52.2), ('AU9', 61.6), ('AU10', 49.05), ('AU12', 61.82), ('AU14', 68.54), ('AU15', 47.5), ('AU17', 68.46)]

MultiLabelF1Score
Dataset: casme, n=189 | train_mean: 0.9366 | test_mean: 0.3057
Test per AU: [('AU1', 65.12), ('AU2', 64.29), ('AU4', 68.53), ('AU5', 0.0), ('AU6', 0.0), ('AU7', 12.5), ('AU9', 33.96), ('AU10', 6.45), ('AU12', 30.3), ('AU14', 43.24), ('AU15', 0.0), ('AU17', 42.42)]



100%|███████████████████████████████████████████| 40/40 [21:28<00:00, 32.22s/it]


MultiLabelF1Score
Dataset: casme2, n=256 | train_mean: 0.957 | test_mean: 0.64
Test per AU: [('AU1', 87.16), ('AU2', 90.45), ('AU4', 91.79), ('AU5', 49.51), ('AU6', 48.7), ('AU7', 47.45), ('AU9', 51.03), ('AU10', 53.94), ('AU12', 63.19), ('AU14', 77.94), ('AU15', 48.39), ('AU17', 58.44)]

MultiLabelF1Score
Dataset: casme2, n=256 | train_mean: 0.9171 | test_mean: 0.335
Test per AU: [('AU1', 76.92), ('AU2', 82.61), ('AU4', 91.63), ('AU5', 0.0), ('AU6', 0.0), ('AU7', 4.35), ('AU9', 20.62), ('AU10', 11.11), ('AU12', 32.56), ('AU14', 60.71), ('AU15', 0.0), ('AU17', 21.43)]



100%|███████████████████████████████████████████| 40/40 [22:13<00:00, 33.34s/it]


...

MultiLabelF1Score
Dataset: fourd, n=267 | train_mean: 0.9607 | test_mean: 0.5744
Test per AU: [('AU1', 51.69), ('AU2', 51.64), ('AU4', 60.31), ('AU5', 49.92), ('AU6', 47.34), ('AU7', 55.94), ('AU9', 52.91), ('AU10', 61.93), ('AU12', 60.63), ('AU14', 83.05), ('AU15', 49.43), ('AU17', 64.52)]

MultiLabelF1Score
Dataset: fourd, n=267 | train_mean: 0.925 | test_mean: 0.2446
Test per AU: [('AU1', 12.24), ('AU2', 13.79), ('AU4', 41.43), ('AU5', 10.53), ('AU6', 0.0), ('AU7', 48.18), ('AU9', 9.52), ('AU10', 25.0), ('AU12', 35.42), ('AU14', 66.67), ('AU15', 0.0), ('AU17', 30.77)]



100%|███████████████████████████████████████████| 40/40 [21:39<00:00, 32.49s/it]


MultiLabelF1Score
Dataset: mmew, n=300 | train_mean: 0.9349 | test_mean: 0.587
Test per AU: [('AU1', 82.74), ('AU2', 80.04), ('AU4', 56.05), ('AU5', 56.86), ('AU6', 49.15), ('AU7', 52.07), ('AU9', 49.24), ('AU10', 46.62), ('AU12', 54.19), ('AU14', 66.03), ('AU15', 49.32), ('AU17', 62.07)]

MultiLabelF1Score
Dataset: mmew, n=300 | train_mean: 0.8754 | test_mean: 0.2442
Test per AU: [('AU1', 71.29), ('AU2', 66.67), ('AU4', 31.58), ('AU5', 26.67), ('AU6', 0.0), ('AU7', 13.56), ('AU9', 0.0), ('AU10', 0.0), ('AU12', 17.24), ('AU14', 37.5), ('AU15', 0.0), ('AU17', 28.57)]



100%|███████████████████████████████████████████| 40/40 [16:15<00:00, 24.38s/it]


MultiLabelF1Score
Dataset: casme3a, n=860 | train_mean: 0.9488 | test_mean: 0.5076
Test per AU: [('AU1', 50.06), ('AU2', 50.37), ('AU4', 63.03), ('AU5', 50.8), ('AU6', 46.65), ('AU7', 44.65), ('AU9', 46.92), ('AU10', 42.74), ('AU12', 50.16), ('AU14', 60.18), ('AU15', 48.19), ('AU17', 55.32)]

MultiLabelF1Score
Dataset: casme3a, n=860 | train_mean: 0.9049 | test_mean: 0.137
Test per AU: [('AU1', 9.7), ('AU2', 8.7), ('AU4', 53.31), ('AU5', 17.39), ('AU6', 0.0), ('AU7', 7.19), ('AU9', 11.31), ('AU10', 4.26), ('AU12', 3.57), ('AU14', 32.58), ('AU15', 2.11), ('AU17', 14.29)]

Final results

MultiLabelF1Score
All AUs:  [('AU1', 66.07), ('AU2', 65.33), ('AU4', 66.33), ('AU5', 53.52), ('AU6', 47.92), ('AU7', 53.98), ('AU9', 52.18), ('AU10', 47.39), ('AU12', 59.46), ('AU14', 65.03), ('AU15', 48.81), ('AU17', 61.04)]
Mean:  57.26


MultiLabelF1Score
All AUs:  [('AU1', 39.14), ('AU2', 37.34), ('AU4', 56.36), ('AU5', 17.51), ('AU6', 0.0), ('AU7', 24.23), ('AU9', 15.38), ('AU10', 4.7), ('AU12', 25.

In [200]:
SCAValidator(Config).validate_n_times(df, input_data, n_times=5)

  0%|                                                     | 0/5 [00:00<?, ?it/s]

MultiLabelF1Score
Dataset: casme, n=189 | train_mean: 0.9538 | test_mean: 0.6007
Test per AU: [('AU1', 77.63), ('AU2', 83.19), ('AU4', 74.21), ('AU5', 48.36), ('AU6', 49.06), ('AU7', 55.03), ('AU9', 48.29), ('AU10', 47.35), ('AU12', 52.62), ('AU14', 74.28), ('AU15', 47.65), ('AU17', 63.16)]

MultiLabelF1Score
Dataset: casme, n=189 | train_mean: 0.9117 | test_mean: 0.2708
Test per AU: [('AU1', 60.0), ('AU2', 68.97), ('AU4', 66.15), ('AU5', 0.0), ('AU6', 0.0), ('AU7', 16.0), ('AU9', 8.89), ('AU10', 0.0), ('AU12', 17.02), ('AU14', 54.55), ('AU15', 0.0), ('AU17', 33.33)]

MultiLabelF1Score
Dataset: casme2, n=256 | train_mean: 0.9559 | test_mean: 0.5899
Test per AU: [('AU1', 76.11), ('AU2', 79.18), ('AU4', 89.45), ('AU5', 49.21), ('AU6', 48.59), ('AU7', 45.53), ('AU9', 53.0), ('AU10', 54.37), ('AU12', 47.79), ('AU14', 68.88), ('AU15', 48.39), ('AU17', 47.43)]

MultiLabelF1Score
Dataset: casme2, n=256 | train_mean: 0.9154 | test_mean: 0.2401
Test per AU: [('AU1', 56.52), ('AU2', 61.54), ('AU

 20%|███████▍                             | 1/5 [3:27:02<13:48:09, 12422.34s/it]

MultiLabelF1Score
Dataset: casme, n=189 | train_mean: 0.9647 | test_mean: 0.5907
Test per AU: [('AU1', 76.38), ('AU2', 75.22), ('AU4', 75.94), ('AU5', 48.5), ('AU6', 47.21), ('AU7', 45.38), ('AU9', 58.07), ('AU10', 52.49), ('AU12', 53.53), ('AU14', 59.59), ('AU15', 52.9), ('AU17', 63.65)]

MultiLabelF1Score
Dataset: casme, n=189 | train_mean: 0.9329 | test_mean: 0.2581
Test per AU: [('AU1', 57.14), ('AU2', 53.85), ('AU4', 70.34), ('AU5', 0.0), ('AU6', 0.0), ('AU7', 0.0), ('AU9', 27.45), ('AU10', 10.0), ('AU12', 19.61), ('AU14', 25.81), ('AU15', 10.53), ('AU17', 35.0)]

MultiLabelF1Score
Dataset: casme2, n=256 | train_mean: 0.9527 | test_mean: 0.6281
Test per AU: [('AU1', 82.43), ('AU2', 87.51), ('AU4', 85.79), ('AU5', 59.2), ('AU6', 48.7), ('AU7', 49.84), ('AU9', 58.9), ('AU10', 48.39), ('AU12', 63.48), ('AU14', 73.69), ('AU15', 48.39), ('AU17', 47.43)]

MultiLabelF1Score
Dataset: casme2, n=256 | train_mean: 0.9091 | test_mean: 0.3086
Test per AU: [('AU1', 68.09), ('AU2', 76.92), ('AU4

 40%|███████████████▏                      | 2/5 [5:52:49<8:33:02, 10260.72s/it]

...

MultiLabelF1Score
Dataset: casme, n=189 | train_mean: 0.97 | test_mean: 0.6135
Test per AU: [('AU1', 74.84), ('AU2', 81.9), ('AU4', 76.94), ('AU5', 47.79), ('AU6', 48.5), ('AU7', 55.62), ('AU9', 63.85), ('AU10', 45.53), ('AU12', 54.25), ('AU14', 71.27), ('AU15', 47.79), ('AU17', 67.88)]

MultiLabelF1Score
Dataset: casme, n=189 | train_mean: 0.9428 | test_mean: 0.3008
Test per AU: [('AU1', 55.0), ('AU2', 66.67), ('AU4', 71.33), ('AU5', 0.0), ('AU6', 0.0), ('AU7', 18.75), ('AU9', 38.6), ('AU10', 0.0), ('AU12', 21.82), ('AU14', 48.78), ('AU15', 0.0), ('AU17', 40.0)]

MultiLabelF1Score
Dataset: casme2, n=256 | train_mean: 0.9484 | test_mean: 0.6424
Test per AU: [('AU1', 83.72), ('AU2', 87.16), ('AU4', 91.01), ('AU5', 48.49), ('AU6', 48.59), ('AU7', 52.13), ('AU9', 57.79), ('AU10', 48.18), ('AU12', 64.81), ('AU14', 68.26), ('AU15', 48.28), ('AU17', 72.47)]

MultiLabelF1Score
Dataset: casme2, n=256 | train_mean: 0.9009 | test_mean: 0.3391
Test per AU: [('AU1', 71.19), ('AU2', 76.92), ('AU4',

 80%|██████████████████████████████▍       | 4/5 [10:42:11<2:33:18, 9198.25s/it]

MultiLabelF1Score
Dataset: casme, n=189 | train_mean: 0.9732 | test_mean: 0.5928
Test per AU: [('AU1', 75.7), ('AU2', 75.72), ('AU4', 69.55), ('AU5', 49.87), ('AU6', 48.64), ('AU7', 54.13), ('AU9', 47.76), ('AU10', 46.91), ('AU12', 53.95), ('AU14', 64.89), ('AU15', 59.96), ('AU17', 64.26)]

MultiLabelF1Score
Dataset: casme, n=189 | train_mean: 0.9491 | test_mean: 0.2594
Test per AU: [('AU1', 56.41), ('AU2', 55.17), ('AU4', 61.97), ('AU5', 0.0), ('AU6', 0.0), ('AU7', 14.81), ('AU9', 8.51), ('AU10', 0.0), ('AU12', 16.67), ('AU14', 36.84), ('AU15', 25.0), ('AU17', 35.9)]

MultiLabelF1Score
Dataset: casme2, n=256 | train_mean: 0.9563 | test_mean: 0.6218
Test per AU: [('AU1', 84.72), ('AU2', 89.04), ('AU4', 87.5), ('AU5', 55.36), ('AU6', 48.7), ('AU7', 49.02), ('AU9', 52.65), ('AU10', 48.07), ('AU12', 53.03), ('AU14', 69.51), ('AU15', 53.94), ('AU17', 54.66)]

MultiLabelF1Score
Dataset: casme2, n=256 | train_mean: 0.9165 | test_mean: 0.3
Test per AU: [('AU1', 72.73), ('AU2', 80.0), ('AU4', 

100%|████████████████████████████████████████| 5/5 [13:06:41<00:00, 9440.21s/it]

MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
67.5 & 68.2 & 66.7 & 52.0 & 48.0 & 53.3 & 52.2 & 46.7 & 56.8 & 63.9 & 50.8 & 59.6 & 57.1

Datasets:  ['casme', 'casme2', 'samm', 'fourd', 'mmew', 'casme3a', 'Average']
59.9 & 61.9 & 55.4 & 54.0 & 59.4 & 51.6 & 57.0
MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
42.3 & 42.8 & 56.2 & 14.8 & 1.2 & 23.4 & 13.4 & 3.0 & 21.8 & 37.1 & 4.0 & 22.9 & 23.6

Datasets:  ['casme', 'casme2', 'samm', 'fourd', 'mmew', 'casme3a', 'Average']
27.0 & 29.3 & 19.2 & 17.2 & 25.7 & 16.0 & 22.4


In [189]:
out = SCAValidator(Config).validate(df, input_data)

100%|███████████████████████████████████████████| 40/40 [04:11<00:00,  6.28s/it]
/home/tvaranka/anaconda3/envs/python3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1599: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/home/tvaranka/anaconda3/envs/python3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1599: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/home/tvaranka/anaconda3/envs/python3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1599: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_di

MultiLabelF1Score
Subject: 01, n=9 | train_mean: 0.8517 | test_mean: 0.5524
Test per AU: [('AU1', 100.0), ('AU2', 100.0), ('AU4', 43.75), ('AU5', 50.0), ('AU6', 100.0), ('AU7', 10.0), ('AU9', 0.0), ('AU10', 25.0), ('AU12', 47.06), ('AU14', 100.0), ('AU15', 40.0), ('AU17', 47.06)]

MultiLabelF1Score
Subject: 01, n=9 | train_mean: 0.7354 | test_mean: 0.1007
Test per AU: [('AU1', 0.0), ('AU2', 0.0), ('AU4', 87.5), ('AU5', 33.33), ('AU6', 0.0), ('AU7', 0.0), ('AU9', 0.0), ('AU10', 0.0), ('AU12', 0.0), ('AU14', 0.0), ('AU15', 0.0), ('AU17', 0.0)]



100%|███████████████████████████████████████████| 40/40 [04:12<00:00,  6.30s/it]


MultiLabelF1Score
Subject: 02, n=13 | train_mean: 0.8477 | test_mean: 0.3375
Test per AU: [('AU1', 62.86), ('AU2', 45.83), ('AU4', 48.0), ('AU5', 23.53), ('AU6', 27.78), ('AU7', 18.75), ('AU9', 38.1), ('AU10', 27.78), ('AU12', 22.62), ('AU14', 38.1), ('AU15', 29.09), ('AU17', 22.62)]

MultiLabelF1Score
Subject: 02, n=13 | train_mean: 0.7275 | test_mean: 0.15
Test per AU: [('AU1', 40.0), ('AU2', 0.0), ('AU4', 0.0), ('AU5', 0.0), ('AU6', 0.0), ('AU7', 0.0), ('AU9', 0.0), ('AU10', 0.0), ('AU12', 28.57), ('AU14', 42.86), ('AU15', 40.0), ('AU17', 28.57)]



100%|███████████████████████████████████████████| 40/40 [04:13<00:00,  6.33s/it]
/home/tvaranka/anaconda3/envs/python3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1599: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/home/tvaranka/anaconda3/envs/python3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1599: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/home/tvaranka/anaconda3/envs/python3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1599: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_di

...

MultiLabelF1Score
Subject: 24, n=11 | train_mean: 0.8629 | test_mean: 0.5837
Test per AU: [('AU1', 38.89), ('AU2', 47.62), ('AU4', 31.25), ('AU5', 26.67), ('AU6', 47.62), ('AU7', 38.89), ('AU9', 15.38), ('AU10', 54.17), ('AU12', 100.0), ('AU14', 100.0), ('AU15', 100.0), ('AU17', 100.0)]

MultiLabelF1Score
Subject: 24, n=11 | train_mean: 0.7528 | test_mean: 0.07986
Test per AU: [('AU1', 0.0), ('AU2', 0.0), ('AU4', 62.5), ('AU5', 0.0), ('AU6', 0.0), ('AU7', 0.0), ('AU9', 0.0), ('AU10', 33.33), ('AU12', 0.0), ('AU14', 0.0), ('AU15', 0.0), ('AU17', 0.0)]



100%|███████████████████████████████████████████| 40/40 [04:13<00:00,  6.34s/it]
/home/tvaranka/anaconda3/envs/python3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1599: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/home/tvaranka/anaconda3/envs/python3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1599: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


MultiLabelF1Score
Subject: 25, n=7 | train_mean: 0.8499 | test_mean: 0.4814
Test per AU: [('AU1', 100.0), ('AU2', 65.0), ('AU4', 57.14), ('AU5', 36.36), ('AU6', 100.0), ('AU7', 65.0), ('AU9', 46.15), ('AU10', 36.36), ('AU12', 0.0), ('AU14', 0.0), ('AU15', 41.67), ('AU17', 30.0)]

MultiLabelF1Score
Subject: 25, n=7 | train_mean: 0.7297 | test_mean: 0.131
Test per AU: [('AU1', 0.0), ('AU2', 50.0), ('AU4', 57.14), ('AU5', 0.0), ('AU6', 0.0), ('AU7', 50.0), ('AU9', 0.0), ('AU10', 0.0), ('AU12', 0.0), ('AU14', 0.0), ('AU15', 0.0), ('AU17', 0.0)]



100%|███████████████████████████████████████████| 40/40 [04:09<00:00,  6.23s/it]
/home/tvaranka/anaconda3/envs/python3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1599: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


MultiLabelF1Score
Subject: 26, n=17 | train_mean: 0.8517 | test_mean: 0.4766
Test per AU: [('AU1', 48.48), ('AU2', 48.48), ('AU4', 70.59), ('AU5', 15.0), ('AU6', 100.0), ('AU7', 46.88), ('AU9', 46.88), ('AU10', 39.29), ('AU12', 40.97), ('AU14', 23.53), ('AU15', 43.33), ('AU17', 48.48)]

MultiLabelF1Score
Subject: 26, n=17 | train_mean: 0.7376 | test_mean: 0.1097
Test per AU: [('AU1', 0.0), ('AU2', 0.0), ('AU4', 70.59), ('AU5', 0.0), ('AU6', 0.0), ('AU7', 0.0), ('AU9', 0.0), ('AU10', 0.0), ('AU12', 37.5), ('AU14', 23.53), ('AU15', 0.0), ('AU17', 0.0)]

Final results

MultiLabelF1Score
All AUs:  [('AU1', 51.63), ('AU2', 57.96), ('AU4', 73.78), ('AU5', 43.43), ('AU6', 50.39), ('AU7', 48.84), ('AU9', 44.04), ('AU10', 40.84), ('AU12', 50.0), ('AU14', 45.61), ('AU15', 50.12), ('AU17', 55.55)]
Mean:  51.02


MultiLabelF1Score
All AUs:  [('AU1', 23.85), ('AU2', 28.57), ('AU4', 72.65), ('AU5', 2.78), ('AU6', 16.87), ('AU7', 18.87), ('AU9', 5.13), ('AU10', 2.25), ('AU12', 31.25), ('AU14', 24.36)

In [190]:
class SCAValidator(core.CrossDatasetValidator):
    def __init__(self, config: Config, verbose: bool = True):
        super().__init__(config)
        self.verbose = verbose
        
    def setup_training(self) -> None:
        """Setup training components

        Sets up the training modules, including model, criterion, optimizer, scheduler
        and mixup. This function can be overwritten to setup custom objects.
        """
        self.model = self.cf.model()
        self.criterion = self.cf.criterion()
        self.model.to(self.cf.device)
        params_for_optimizer = get_fine_tuning_parameters(self.model, 4)
        self.optimizer = self.cf.optimizer(params_for_optimizer)
        self.scheduler = (
            self.cf.scheduler(self.optimizer) if self.cf.scheduler else None
        )
        self.mixup_fn = self.cf.mixup_fn() if self.cf.mixup_fn else None
        
    def train_model(
        self,
        train_loader: torch.utils.data.DataLoader,
        test_loader: torch.utils.data.DataLoader,
    ) -> None:
        """Main training loop

        The main training loop that controls the scheduler and printing.
        This function can be overwritten for custom schedulers, printing,
        etc.
        """
        for epoch in tqdm(range(self.cf.epochs), disable=self.disable_tqdm):
            self.train_one_epoch(epoch, train_loader)
            for param_group in self.optimizer.param_groups:
                param_group['lr'] = 0.001 * (0.1 ** (epoch // 30))
            if self.cf.validation_interval:
                if (epoch + 1) % self.cf.validation_interval == 0:
                    train_metrics = self.evaluate_model(train_loader)
                    test_metrics, outputs_test = self.evaluate_model(
                        test_loader, test=True
                    )
                    self.printer.print_train_test_validation(
                        train_metrics, test_metrics, epoch
                    )
    